In [37]:
import numpy as np
import pandas as pd
import json
from datetime import datetime
from collections import defaultdict
import random

def time_to_minutes(t):
    try:
        dt = datetime.strptime(t, "%H:%M:%S")
    except ValueError:
        dt = datetime.strptime(t, "%H:%M")
    return dt.hour * 60 + dt.minute

# Inisialisasi Data
dosen_df = pd.read_csv('tbl_dosen_rows.csv')
mk_genap_df = pd.read_csv('tbl_mk_genap_rows.csv')
data_dosen_df = pd.read_csv('tbl_data_dosen_rows.csv')
preferensi_dosen_df = pd.read_csv('tbl_preferensi_dosen_rows.csv')
preferensi_prodi_df = pd.read_csv('tbl_preferensi_prodi_rows.csv')
hari_df = pd.read_csv('tbl_hari_rows.csv')
ruang_df = pd.read_csv('tbl_ruang_rows.csv')
jam_df = pd.read_csv('tbl_jam_rows.csv')

# Urutkan jam_df sebelum generate slot
jam_df = jam_df.sort_values('id_jam')

# Gabungkan data menggunakan merge
merged_df = pd.merge(
    pd.merge(data_dosen_df, dosen_df, on='id_dosen'),
    mk_genap_df, on='id_mk_genap'
)

# Tambahkan temporary id secara unik untuk setiap baris di merged_df
merged_df['temp_id'] = range(1, len(merged_df) + 1)

Tahapan GWO

1. Preprocessing
    
    Membangun struktur slot waktu.

2. Inisialisasi Populasi (GWO)
    

    Inisialisasi populasi serigala acak (Xi).

    Setiap "serigala" dalam GWO mewakili solusi penjadwalan yang mungkin (misalnya: variasi pengaturan slot).

3. Fitness Function (GWO)
    Hitung nilai fitness untuk setiap serigala.

    Mengevaluasi kualitas penjadwalan (misalnya: minimalisasi konflik, kepadatan ruang, dll).

4. Proses Optimasi (GWO)
    Tentukan Alpha, Beta, dan Delta berdasarkan nilai fitness.

    Menggunakan hierarki Alpha, Beta, Delta untuk memperbarui posisi solusi.

    While (iterasi < maksimum iterasi):
      1. Perbarui parameter a, A, dan C.
      2. For setiap serigala:
          1. Perbarui posisi serigala berdasarkan Alpha, Beta, dan Delta.
      3. Hitung nilai fitness untuk setiap serigala.
      4. Perbarui Alpha, Beta, dan Delta.
5. Postprocessing
    Return solusi terbaik (Alpha).
    
    Menampilkan jadwal terbaik hasil optimasi.


In [38]:
# @title Preprosessing
# Generator slot jadwal
semester_genap = [2, 4, 6, 8]
semester_ganjil = [1, 3, 5, 7]

def slot_generator():
    slots = []
    id_counter = 1
    for semester in semester_genap:
        for hari in hari_df['nama_hari']:
            for ruang in ruang_df['nama_ruang']:
                for jam in jam_df.itertuples():
                    slots.append({
                        "id_slot": id_counter,
                        "id_mk": None,
                        "mata_kuliah": None,
                        "id_dosen": None,
                        "dosen": None,
                        "ruang": ruang,
                        "hari": hari,
                        "jam_mulai": jam.jam_awal,
                        "jam_selesai": jam.jam_akhir,
                        "semester": semester,
                        "kelas": None,
                        "sks": None,
                        "metode": None,
                        "status": None,
                        "temp_id": None
                    })
                    id_counter += 1
    return slots

In [39]:
def create_random_schedule():
    schedule = slot_generator()
    merged_shuffled = merged_df.sort_values(by='sks', ascending=False).iterrows()

    for _, row in merged_shuffled:
        id_mk = row['id_mk_genap']
        mata_kuliah = row['nama_mk_genap']
        id_dosen = row['id_dosen']
        dosen = row['nama_dosen']
        kelas = row['kelas']
        sks = int(row['sks'])
        original_semester = row['smt']
        
        adjusted_semester = original_semester + 1 if original_semester % 2 != 0 else original_semester
        
        metode = row['metode']
        temp_id = row['temp_id']

        possible_positions = []
        for i in range(len(schedule) - sks + 1):
            block = schedule[i:i + sks]
            if all(
            slot['mata_kuliah'] is None and
            slot['hari'] == block[0]['hari'] and
            slot['ruang'] == block[0]['ruang'] and
            slot['semester'] == adjusted_semester 
            for slot in block):
                possible_positions.append(block)

        if not possible_positions:
            continue
        
        random.shuffle(possible_positions)
        for block in possible_positions:
                for slot in block:
                    slot.update({
                        "id_mk": id_mk,
                        "mata_kuliah": mata_kuliah,
                        "id_dosen": id_dosen,
                        "dosen": dosen,
                        "kelas": kelas,
                        "sks": sks,
                        "semester": adjusted_semester,
                        "metode": metode,
                        "temp_id": temp_id
                    })
                break
        else:
            print(f"Gagal menempatkan: {kelas} - {mata_kuliah} - {dosen} - Semester {adjusted_semester}")
            return 400, "gagal menempatkan mata kuliah"
    return schedule

# Contoh penggunaan:
population_size = 1
population = [create_random_schedule() for _ in range(population_size)]

# Uncomment untuk menampilkan jadwal
i=0
for schedule in population:
    for slot in schedule:
        print(slot)
        if slot['mata_kuliah'] is not None:
            i+=1
        # print(slot)
    if i == merged_df.sum()['sks']:
        print("Jadwal Sudah Lengkap")
    else:
        print("Jadwal Belum Lengkap")

print("Jumlah slot yang terisi: ", i)


{'id_slot': 1, 'id_mk': None, 'mata_kuliah': None, 'id_dosen': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'semester': 2, 'kelas': None, 'sks': None, 'metode': None, 'status': None, 'temp_id': None}
{'id_slot': 2, 'id_mk': None, 'mata_kuliah': None, 'id_dosen': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'semester': 2, 'kelas': None, 'sks': None, 'metode': None, 'status': None, 'temp_id': None}
{'id_slot': 3, 'id_mk': 211820631, 'mata_kuliah': 'Matematika Diskrit', 'id_dosen': 11, 'dosen': 'Dewi Soyusiawaty, S.T., M.T', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'semester': 2, 'kelas': 'E', 'sks': 3, 'metode': 'Offline', 'status': None, 'temp_id': 39}
{'id_slot': 4, 'id_mk': 211820631, 'mata_kuliah': 'Matematika Diskrit', 'id_dosen': 11, 'dosen': 'Dewi Soyusiawaty, S.T., M.T', 'ruang': '4.1.5.55', 'hari': 'Senin',

Konstrain berat 1
Konstrain ringan 0.5

Konstrain berat:
dosen tidak boleh mengajar mata kuliah/kelas berbeda pada jam yang sama
ruang kelas tidak boleh digunakan lebih dari 1 kelas/dosen
3 sks membutuhkan 3 slot waktu
ruangan antar kelas harus sama
slot waktu harus berurutan
jika kelas sama, semester juga sama, maka tidak boleh mulai di waktu yang sama

Konstrain ringan: pereferensi dosen
dosen Ardiansyah, Dr., S.T., M.Cs. Tidak ingin kelas sebelum 12:00 PM
dosen Ali Tarmuji, S.T., M.Cs. Tidak ingin ada kelas pada hari Sabtu
dosen Bambang Robiin, S.T., M.T. tidak ingin kelas setelah 12:00 PM
dosen Tedy Setiadi, Drs., M.T. Tidak ingin ada kelas pada hari Sabtu dan Kamis

In [40]:
# Fungsi untuk mengambil konfigurasi preferensi dosen
from collections import defaultdict
import numpy as np

# Mapping dictionaries
hari_map = dict(zip(hari_df["id_hari"], hari_df["nama_hari"]))
jam_awal_map = dict(zip(jam_df["id_jam"], jam_df["jam_awal"]))
jam_akhir_map = dict(zip(jam_df["id_jam"], jam_df["jam_akhir"]))
dosen_map = dict(zip(dosen_df["id_dosen"], dosen_df["nama_dosen"]))

# Convert preferences for lecturers
def get_lecturer_preferences():
    lecturer_preferences = {}

    for _, row in preferensi_dosen_df.iterrows():
        id_dosen = int(row["dosen_id"])
        nama_dosen = dosen_map.get(id_dosen, f"Dosen {id_dosen}")

        # Parse day preference
        hari_raw = row["hari"]
        if pd.isna(hari_raw):
            restricted_days = []
        elif isinstance(hari_raw, str) and hari_raw.startswith("["):
            hari_list = [int(h.strip()) for h in hari_raw.strip("[]").split(",")]
            restricted_days = [hari_map[h] for h in hari_list if h in hari_map]
        elif isinstance(hari_raw, (int, float, np.integer, np.floating)):
            restricted_days = [hari_map[int(hari_raw)]] if int(hari_raw) in hari_map else []
        else:
            restricted_days = []

        # Parse time range
        jam_mulai_id = row["jam_mulai_id"]
        jam_selesai_id = row["jam_selesai_id"]
        if pd.notna(jam_mulai_id) and pd.notna(jam_selesai_id):
            start = jam_awal_map.get(int(jam_mulai_id))
            end = jam_akhir_map.get(int(jam_selesai_id))
            time_range = [start, end] if start and end else []
        else:
            time_range = []

        # Assign formatted preference
        lecturer_preferences[nama_dosen] = {
            "restricted_days": restricted_days,
            "time_range": time_range
        }

    return lecturer_preferences

# Convert preferences for program (prodi)
def get_prodi_preferences():
    prodi_pref = {
        "restricted_days": [],
        "restricted_time_ranges": []
    }

    for _, row in preferensi_prodi_df.iterrows():
        # Parse days
        hari_raw = row["hari"]
        if isinstance(hari_raw, (int, float, np.integer, np.floating)):
            hari_nama = hari_map.get(int(hari_raw))
            if hari_nama and hari_nama not in prodi_pref["restricted_days"]:
                prodi_pref["restricted_days"].append(hari_nama)

        # Parse time range
        jam_mulai_id = row["jam_mulai_id"]
        jam_selesai_id = row["jam_selesai_id"]
        if pd.notna(jam_mulai_id) and pd.notna(jam_selesai_id):
            start = jam_awal_map.get(int(jam_mulai_id))
            end = jam_akhir_map.get(int(jam_selesai_id))
            if start and end:
                prodi_pref["restricted_time_ranges"].append((start, end))

    return prodi_pref

# Cetak hasil
print("Preferensi Dosen:", get_lecturer_preferences())
print("Preferensi Prodi:", get_prodi_preferences())

def collect_conflicts(schedule, prodi_id=None):
    conflict_temp_ids = set()
    lecturer_preferences = get_lecturer_preferences()
    prodi_prefs = get_prodi_preferences()

    room_consistency_conflicts = []
    semester_consistency_conflicts = []
    teacher_conflicts = []
    room_conflicts = []
    class_conflicts = []
    lecturer_preference_conflicts = []
    prodi_preference_conflicts = []
    preference_conflict_temp_ids = set()

    # (A) Konsistensi Ruangan dan Semester
    temp_groups = defaultdict(list)
    for slot in schedule:
        tid = slot.get('temp_id')
        if slot['mata_kuliah'] and tid is not None:
            temp_groups[tid].append(slot)
    for tid, slots in temp_groups.items():
        rooms = {slot['ruang'] for slot in slots}
        semesters = {slot.get('semester') for slot in slots}
        if len(rooms) > 1:
            conflict_temp_ids.add(tid)
            room_consistency_conflicts.append({
                'temp_id': tid,
                'ruangan': list(rooms),
                'slot_ids': [slot['id_slot'] for slot in slots]
            })
        if len(semesters) > 1:
            conflict_temp_ids.add(tid)
            semester_consistency_conflicts.append({
                'temp_id': tid,
                'semesters': list(semesters),
                'slot_ids': [slot['id_slot'] for slot in slots]
            })

    # (B) Konflik Dosen
    teacher_groups = defaultdict(list)
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
        teacher_groups[(slot['dosen'], slot['hari'].lower())].append(slot)
    for (dosen, hari), slots in teacher_groups.items():
        slots.sort(key=lambda s: time_to_minutes(s['jam_mulai']))
        for i in range(len(slots)):
            for j in range(i+1, len(slots)):
                s1, s2 = slots[i], slots[j]
                if time_to_minutes(s2['jam_mulai']) < time_to_minutes(s1['jam_selesai']) and s1['mata_kuliah'] != s2['mata_kuliah']:
                    for s in (s1, s2):
                        tid = s.get('temp_id')
                        if tid is not None:
                            conflict_temp_ids.add(tid)
                    teacher_conflicts.append({
                        'dosen': dosen,
                        'hari': hari,
                        'slot_ids': [s1['id_slot'], s2['id_slot']]
                    })

    # (C) Konflik Ruangan
    room_groups = defaultdict(list)
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
        if slot['metode'] == "Online":
            continue
        room_groups[(slot['ruang'], slot['hari'].lower())].append(slot)
    for (ruang, hari), slots in room_groups.items():
        slots.sort(key=lambda s: time_to_minutes(s['jam_mulai']))
        for i in range(len(slots)):
            for j in range(i+1, len(slots)):
                s1, s2 = slots[i], slots[j]
                if time_to_minutes(s2['jam_mulai']) <= time_to_minutes(s1['jam_selesai']) and s1['kelas'] != s2['kelas']:
                    for s in (s1, s2):
                        tid = s.get('temp_id')
                        if tid is not None:
                            conflict_temp_ids.add(tid)
                    room_conflicts.append({
                        'ruang': ruang,
                        'hari': hari,
                        'slot_ids': [s1['id_slot'], s2['id_slot']]
                    })

    # (D) Konflik Preferensi Dosen
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
        tid = slot.get('temp_id')
        if tid is None:
            continue
        dosen = slot['dosen']
        hari = slot['hari'].lower()
        start = time_to_minutes(slot['jam_mulai'])

        if dosen not in lecturer_preferences:
            continue

        prefs = lecturer_preferences[dosen]
        for pref in prefs:
            if isinstance(pref, list):  # preferensi hari (daftar hari yang tidak boleh)
                if hari in pref:
                    lecturer_preference_conflicts.append({
                        'temp_id': tid,
                        'type': 'day',
                        'dosen': dosen,
                        'restricted_days': pref,
                        'slot_id': slot['id_slot']
                    })
                    preference_conflict_temp_ids.add(tid)
                    break
            elif isinstance(pref, dict):  # preferensi waktu
                jam_mulai_id = pref.get('jam_mulai_id')
                jam_selesai_id = pref.get('jam_selesai_id')
                if jam_mulai_id and jam_selesai_id:
                    allowed_start = time_to_minutes(jam_mulai_id)
                    allowed_end = time_to_minutes(jam_selesai_id)
                    if not (allowed_start <= start < allowed_end):
                        lecturer_preference_conflicts.append({
                            'temp_id': tid,
                            'type': 'time',
                            'dosen': dosen,
                            'allowed': (jam_mulai_id, jam_selesai_id),
                            'slot_id': slot['id_slot']
                        })
                        preference_conflict_temp_ids.add(tid)
                        break

    # (E) Konflik Kelas
    class_groups = defaultdict(list)
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
        class_groups[(slot['kelas'], slot['semester'], slot['hari'].lower())].append(slot)

    for key, slots in class_groups.items():
        # Hanya periksa bentrok jika semester dan kelas sama (berarti angkatan sama)
        slots.sort(key=lambda s: time_to_minutes(s['jam_mulai']))
        for i in range(len(slots)):
            for j in range(i+1, len(slots)):
                s1, s2 = slots[i], slots[j]
                if time_to_minutes(s2['jam_mulai']) < time_to_minutes(s1['jam_selesai']):
                    # Tambahkan validasi: skip jika dosennya berbeda dan ruangnya berbeda
                    if s1['dosen'] != s2['dosen'] and s1['ruang'] != s2['ruang']:
                        continue  # tidak dianggap bentrok
                    for s in (s1, s2):
                        tid = s.get('temp_id')
                        if tid is not None:
                            conflict_temp_ids.add(tid)
                    class_conflicts.append({
                        'kelas': key[0],
                        'hari': key[2],
                        'semester': key[1],
                        'slot_ids': [s1['id_slot'], s2['id_slot']]
                    })

    # (F) Konflik Preferensi Prodi
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
        tid = slot.get('temp_id')
        hari = slot['hari'].lower()
        start = time_to_minutes(slot['jam_mulai'])

        # Global restrictions
        rd = [d.lower() for d in prodi_prefs.get('restricted_days', [])]
        rtr = prodi_prefs.get('restricted_time_ranges', [])
        if hari in rd:
            for tr in rtr:
                rs, re = time_to_minutes(tr[0]), time_to_minutes(tr[1])
                if rs <= start < re:
                    prodi_preference_conflicts.append({
                        'temp_id': tid,
                        'type': 'global',
                        'restricted': tr,
                        'slot_id': slot['id_slot']
                    })
                    preference_conflict_temp_ids.add(tid)
                    break

        # Per-program preferences
        prefs = prodi_prefs.get(prodi_id, [])
        for pref in prefs:
            days = pref.get('hari') or []
            if isinstance(days, (int, str)):
                days = [int(days)] if str(days).isdigit() else []
            if hari in days:
                prodi_preference_conflicts.append({
                    'temp_id': tid,
                    'type': 'day_specific',
                    'slot_id': slot['id_slot'],
                    'restricted_days': days
                })
                preference_conflict_temp_ids.add(tid)
                continue
            if pref.get('jam_mulai_id') and pref.get('jam_selesai_id'):
                as_, ae = time_to_minutes(pref['jam_mulai_id']), time_to_minutes(pref['jam_selesai_id'])
                if start < as_ or start >= ae:
                    prodi_preference_conflicts.append({
                        'temp_id': tid,
                        'type': 'time_specific',
                        'allowed': (pref['jam_mulai_id'], pref['jam_selesai_id']),
                        'slot_id': slot['id_slot']
                    })
                    preference_conflict_temp_ids.add(tid)

    return {
        'conflict_temp_ids': conflict_temp_ids,
        'room_consistency_conflicts': room_consistency_conflicts,
        'semester_consistency_conflicts': semester_consistency_conflicts,
        'teacher_conflicts': teacher_conflicts,
        'room_conflicts': room_conflicts,
        'class_conflicts': class_conflicts,
        'lecturer_preference_conflicts': lecturer_preference_conflicts,
        'preference_conflict_temp_ids': preference_conflict_temp_ids
    }

def calculate_fitness(schedule, db = None):
    conflicts = collect_conflicts(schedule, db)
    penalty = (
        len(conflicts['conflict_temp_ids']) +
        0.5 * len(conflicts['preference_conflict_temp_ids'])
    )
    print(f"Total conflicts: {len(conflicts['conflict_temp_ids'])}, Preference conflicts: {len(conflicts['preference_conflict_temp_ids'])}")
    return penalty

Preferensi Dosen: {'Ardiansyah, Dr., S.T., M.Cs.': {'restricted_days': [], 'time_range': ['12:30:00', '17:50:00']}, 'Ali Tarmuji, S.T., M.Cs.': {'restricted_days': [], 'time_range': ['7:00:00', '17:50:00']}, 'Bambang Robiin, S.T., M.T.': {'restricted_days': [], 'time_range': ['7:00:00', '12:10:00']}, 'Tedy Setiadi, Drs., M.T.': {'restricted_days': ['Kamis', 'Sabtu'], 'time_range': []}, 'Adhi Prahara, S.Si., M.Cs.': {'restricted_days': [], 'time_range': ['7:00:00', '17:50:00']}}
Preferensi Prodi: {'restricted_days': ['Jumat'], 'restricted_time_ranges': [('9:35:00', '13:20:00')]}


In [41]:
import time
def update_position(schedule, alpha, beta, delta, a, collect_conflicts, db, fitness_function):
    new_schedule = [dict(slot) for slot in schedule]

    # Ambil hard dan soft constraints untuk solusi ini
    conflicts = collect_conflicts(new_schedule, db)
    # print(f"Konflik: {conflicts}")
    hard_constraints = conflicts['conflict_temp_ids']
    soft_constraints = conflicts['preference_conflict_temp_ids']

    temp_id_groups = defaultdict(list)
    for slot in new_schedule:
        if slot['temp_id'] is not None:
            temp_id_groups[slot['temp_id']].append(slot)

    def group_by_temp(schedule):
        temp_groups = defaultdict(list)
        for slot in schedule:
            if slot['temp_id'] is not None:
                temp_groups[slot['temp_id']].append(slot)
        return temp_groups

    alpha_groups = group_by_temp(alpha)
    beta_groups = group_by_temp(beta)
    delta_groups = group_by_temp(delta)

    def handle_constraints(constraint_ids, is_soft=False):
        for temp_id in constraint_ids:
            if temp_id not in temp_id_groups:
                continue

            slots_to_move = temp_id_groups[temp_id]
            sks = len(slots_to_move)

            r1 = random.random()
            A1 = 2 * a * r1 - a
            r1 = random.random()
            A2 = 2 * a * r1 - a
            r1= random.random()
            A3 = 2 * a * r1 - a

            if abs(A1) <= abs(A2) and abs(A1) <= abs(A3):
                ref_group = alpha_groups.get(temp_id, [])
            elif abs(A2) <= abs(A3):
                ref_group = beta_groups.get(temp_id, [])
            else:
                ref_group = delta_groups.get(temp_id, [])

            if not ref_group:
                continue

            possible_positions = []
            for i in range(len(new_schedule) - sks + 1):
                block = new_schedule[i:i + sks]
                selected_semester = ref_group[0]['semester']
                if all(
                    slot['mata_kuliah'] is None and
                    slot['hari'] == block[0]['hari'] and
                    slot['ruang'] == block[0]['ruang'] and
                    slot['semester'] == selected_semester and
                    not any(
                        s['id_dosen'] == slots_to_move[0]['id_dosen'] and
                        s['hari'] == slot['hari'] and
                        s['jam_mulai'] == slot['jam_mulai']
                        for s in new_schedule if s['id_dosen'] is not None
                    )
                    for slot in block):
                    possible_positions.append(block)
                    
            if not possible_positions:
                continue

            new_block = random.choice(possible_positions)

            for old_slot, new_slot, ref_slot in zip(slots_to_move, new_block, ref_group):
                new_slot.update({
                    "id_mk": old_slot["id_mk"],
                    "mata_kuliah": old_slot["mata_kuliah"],
                    "id_dosen": old_slot["id_dosen"],
                    "dosen": old_slot["dosen"],
                    "semester": old_slot["semester"],
                    "kelas": old_slot["kelas"],
                    "sks": old_slot["sks"],
                    "metode": old_slot["metode"],
                    "temp_id": old_slot["temp_id"]
                })

                old_slot.update({
                    "id_mk": None, "mata_kuliah": None,
                    "id_dosen": None, "dosen": None, "kelas": None,
                    "sks": None, "metode": None,
                    "temp_id": None
                })

    handle_constraints(hard_constraints)
    handle_constraints(soft_constraints, is_soft=True)

    return new_schedule

class GreyWolfOptimizer:
    def __init__(self, population_size, max_iterations):
        self.population_size = population_size
        self.max_iterations = max_iterations

    def optimize(self, fitness_function, create_solution_function, collect_conflicts):
        start_time = time.time()  # Waktu awal eksperimen
        population = [create_solution_function() for _ in range(self.population_size)]
        fitness_values = [fitness_function(schedule) for schedule in population]
        print(f"Populasi awal: {fitness_values}")

        best_solution = None
        best_fitness = float('inf')
        iteration_times = []

        for iteration in range(self.max_iterations):
            # Tidak pakai iter_start lagi

            sorted_pop = sorted(zip(population, fitness_values), key=lambda x: x[1])
            alpha, alpha_fitness = sorted_pop[0]
            beta, beta_fitness = sorted_pop[1]
            delta, delta_fitness = sorted_pop[2]

            if alpha_fitness < best_fitness:
                best_solution = alpha
                best_fitness = alpha_fitness

            log_message = f"Iterasi {iteration+1}/{self.max_iterations} - Best Fitness: {best_fitness}"
            print(log_message)

            a = 2 * (1 - iteration / self.max_iterations)

            new_population = []
            new_fitness_values = []

            for schedule in population:
                updated_schedule = update_position(schedule, alpha, beta, delta, a, collect_conflicts, db=None, fitness_function=fitness_function)
                new_population.append(updated_schedule)
                new_fitness_values.append(fitness_function(updated_schedule))

            population = new_population
            fitness_values = new_fitness_values

            # Simpan waktu kumulatif jika iterasi ke-5, 10, 15, ...
            if (iteration + 1) in [5, 10, 15, 20, 25, 30]:
                elapsed_cumulative = time.time() - start_time
                iteration_times.append((iteration + 1, elapsed_cumulative))

        total_elapsed = time.time() - start_time

        print("Optimasi Selesai!")
        print(f"Best Fitness: {best_fitness}")

        # Simpan ke Excel
        df_time = pd.DataFrame(iteration_times, columns=["Iterasi", "Waktu Kumulatif (detik)"])
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        df_time.to_excel(f"Waktu Kumulatif Tiap 5 Iterasi_{timestamp}_populasi{population_size}.xlsx", index=False)

        return best_solution, best_fitness, total_elapsed


In [42]:
# if __name__ == "__main__":
#     population_size = 30
#     max_iterations = 30

#     gwo = GreyWolfOptimizer(population_size, max_iterations)

#     best_schedule, best_fitness = gwo.optimize(
#         fitness_function=lambda schedule: calculate_fitness(schedule),
#         create_solution_function=create_random_schedule, 
#         collect_conflicts=collect_conflicts
#         )

#     total_terisi = sum(1 for slot in best_schedule if slot['mata_kuliah'] is not None)
#     print(f"Total slot terisi: {total_terisi}")

#     total_sks = merged_df['sks'].sum()
#     print("Jadwal Sudah Lengkap" if total_terisi == total_sks else "Jadwal Belum Lengkap")

#     with open('output.json', 'w') as f:
#         json.dump(best_schedule, f, indent=4)

In [43]:
if __name__ == "__main__":
    population_sizes = [5, 10, 15, 20, 25, 30]
    max_iterations_list = [30]
    num_experiments = 1

    experiment_data = []

    for pop_size in population_sizes:
        for max_iter in max_iterations_list:
            for experiment in range(num_experiments):
                gwo = GreyWolfOptimizer(pop_size, max_iter)
                best_schedule, best_fitness, elapsed_time = gwo.optimize(
                    fitness_function=calculate_fitness,
                    create_solution_function=create_random_schedule, 
                    collect_conflicts=collect_conflicts
                )
                print(f"Experiment {experiment+1}/{num_experiments} (Population: {pop_size}, Iterations: {max_iter}) - Best Fitness: {best_fitness}")
                experiment_data.append((pop_size, max_iter, elapsed_time))

    grouped_data = defaultdict(list)
    for pop_size, max_iter, fitness in experiment_data:
        grouped_data[(pop_size, max_iter)].append(fitness)
    
    columns = []
    
    population_sizes = sorted(set([pop for pop, _, _ in experiment_data]))
    max_iterations_list = sorted(set([it for _, it, _ in experiment_data]))
    
    for pop_size in population_sizes:
        for max_iter in max_iterations_list:
            # Header kolom
            col_header = [f"Individu {pop_size} Iterasi {max_iter}"]
            
            # Data fitness (konversi ke string dengan koma sebagai desimal)
            fitness_values = [str(f) for f in grouped_data[(pop_size, max_iter)]]
            
            # Gabungkan header dan data
            full_column = col_header + fitness_values
            columns.append(full_column)
    
    # 4. Cari panjang kolom terpanjang
    max_length = max(len(col) for col in columns)
    
    # 5. Samakan panjang semua kolom dengan mengisi string kosong
    for col in columns:
        if len(col) < max_length:
            col += [""] * (max_length - len(col))
    
    # 6. Buat DataFrame dan simpan ke Excel
    df = pd.DataFrame({f"Kombinasi {i+1}": col for i, col in enumerate(columns)})
    
    excel_file = "Proses Time Full Terbaru.xlsx"
    df.to_excel(excel_file, index=False, header=False)
    print(f"File Excel berhasil dibuat: {excel_file}")

Total conflicts: 95, Preference conflicts: 14
Total conflicts: 101, Preference conflicts: 16
Total conflicts: 92, Preference conflicts: 16
Total conflicts: 88, Preference conflicts: 14
Total conflicts: 102, Preference conflicts: 12
Populasi awal: [102.0, 109.0, 100.0, 95.0, 108.0]
Iterasi 1/30 - Best Fitness: 95.0
Total conflicts: 85, Preference conflicts: 11
Total conflicts: 84, Preference conflicts: 6
Total conflicts: 81, Preference conflicts: 8
Total conflicts: 77, Preference conflicts: 7
Total conflicts: 83, Preference conflicts: 9
Iterasi 2/30 - Best Fitness: 80.5
Total conflicts: 83, Preference conflicts: 9
Total conflicts: 73, Preference conflicts: 10
Total conflicts: 87, Preference conflicts: 8
Total conflicts: 67, Preference conflicts: 11
Total conflicts: 75, Preference conflicts: 11
Iterasi 3/30 - Best Fitness: 72.5
Total conflicts: 64, Preference conflicts: 7
Total conflicts: 76, Preference conflicts: 10
Total conflicts: 71, Preference conflicts: 12
Total conflicts: 72, Pref

PermissionError: [Errno 13] Permission denied: 'Proses Time Full Terbaru.xlsx'